In [ ]:
#imports
import pandas as pd
import numpy as np
import itertools 
!pip install pyfpgrowth
import pyfpgrowth
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 1.6 MB 4.7 MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5504 sha256=cedd30e3ad83d308558c620087ffee11de67ab2ce055be8660c5334bb3b53d86
  Stored in directory: /root/.cache/pip/wheels/73/97/4b/f12ac994f6bbb99597396255435824c73ad3916be1e678be55
Successfully built pyfpgrowth
Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/Shared drives/641 project/consol_reddit_emotion.csv')
#keeping columns from green:
data = data[['Angry temax','Fear temax','Happy temax','Sad temax', 'Surprise temax']]
data = data.rename(columns={"Angry temax": "anger", "Fear temax": "fear", "Happy temax": "joy", "Sad temax": "sadness", "Surprise temax": "surprise"})
data['total_tweets'] = data.sum(axis=1)
data['anger'] = data['anger']/data['total_tweets']
data['surprise'] = data['surprise']/data['total_tweets']
data['joy'] = data['joy']/data['total_tweets']
data['sadness'] = data['sadness']/data['total_tweets']
data['fear'] = data['fear']/data['total_tweets']
data = data.drop(columns=['total_tweets'])
data.head()

,anger,fear,joy,sadness,surprise
0,0.047619,0.150579,0.445302,0.146718,0.209781
1,0.022346,0.251397,0.393855,0.142458,0.189944
2,0.038363,0.304348,0.345269,0.120205,0.191816
3,0.032032,0.404404,0.247247,0.154154,0.162162
4,0.001193,0.001193,0.992840,0.003580,0.001193


In [ ]:
#Update dataset for requirements
#Group1: 0 - 0.19 - Low 
#Group2: 0.2 - 0.39 - Mid-Low  -> New: 0.2 - 0.29
#Group3: 0.4 - 0.59 - Mid -> New: 0.3 - 0.39
#Group4: 0.6 - 0.79 - Mid-High #not applicable in this dataset -> New: 0.4 - 0.59
#Group5: 0.8 - 1 - High #not applicable in this dataset -> New: 0.6 - 1
column_names = data.columns.values.tolist()
choices = ['Low','MidLow','Mid','MidHigh','High']

for i in column_names[0:len(column_names)]:
    #conditions = [data[i]<0.2, data[i]<0.4, data[i]<0.6, data[i]<0.8, data[i]<=1]
    conditions = [data[i]<0.2, data[i]<0.3, data[i]<0.4, data[i]<0.6, data[i]<=1]
    new_choices = ["-".join([i,str(j)]) for j in choices]
    data[i] = np.select(conditions, new_choices, default=0)
data
#data.all() - check if any zero remaining

,anger,fear,joy,sadness,surprise
0,anger-Low,fear-Low,joy-MidHigh,sadness-Low,surprise-MidLow
1,anger-Low,fear-MidLow,joy-Mid,sadness-Low,surprise-Low
2,anger-Low,fear-Mid,joy-Mid,sadness-Low,surprise-Low
3,anger-Low,fear-MidHigh,joy-MidLow,sadness-Low,surprise-Low
4,anger-Low,fear-Low,joy-High,sadness-Low,surprise-Low
...,...,...,...,...,...
506,anger-Low,fear-Low,joy-High,sadness-Low,surprise-Low
507,anger-Low,fear-Low,joy-MidHigh,sadness-Low,surprise-MidLow
508,anger-Low,fear-High,joy-MidLow,sadness-Low,surprise-Low
509,anger-Low,fear-Low,joy-MidHigh,sadness-Low,surprise-Low


In [ ]:
data['items'] = ''
data['items'] = data['items'].astype('object')
for i in range(data.shape[0]):
    data.at[i,'items'] = [data['anger'][i], data['surprise'][i], data['joy'][i], data['sadness'][i],  data['fear'][i]]
data = data.drop(['anger', 'surprise','joy','sadness','fear'], axis=1)
data

,items
0,"[anger-Low, surprise-MidLow, joy-MidHigh, sadn..."
1,"[anger-Low, surprise-Low, joy-Mid, sadness-Low..."
2,"[anger-Low, surprise-Low, joy-Mid, sadness-Low..."
3,"[anger-Low, surprise-Low, joy-MidLow, sadness-..."
4,"[anger-Low, surprise-Low, joy-High, sadness-Lo..."
...,...
506,"[anger-Low, surprise-Low, joy-High, sadness-Lo..."
507,"[anger-Low, surprise-MidLow, joy-MidHigh, sadn..."
508,"[anger-Low, surprise-Low, joy-MidLow, sadness-..."
509,"[anger-Low, surprise-Low, joy-MidHigh, sadness..."


In [ ]:
#Association rules
#Use find_frequent_patterns to find patterns in baskets that occur over the support threshold:
patterns = pyfpgrowth.find_frequent_patterns(data['items'].tolist(), 100) #100 is min threshold for support count
#Use generate_association_rules to find patterns that are associated with another with a certain minimum probability:
rules = pyfpgrowth.generate_association_rules(patterns, 0.7) #0.7 is the minimum threshold set for confidence

In [ ]:
len(patterns)

36

In [ ]:
len(rules)

28

In [ ]:
rules

{('anger-Low',): (('sadness-Low',), 0.8972332015810277),
 ('anger-Low', 'fear-Low'): (('sadness-Low',), 0.9073359073359073),
 ('anger-Low', 'fear-Low', 'joy-MidHigh'): (('sadness-Low',),
  0.9530201342281879),
 ('anger-Low', 'fear-Low', 'joy-MidHigh', 'sadness-Low'): (('surprise-Low',),
  0.7183098591549296),
 ('anger-Low', 'fear-Low', 'joy-MidHigh', 'surprise-Low'): (('sadness-Low',),
  0.9532710280373832),
 ('anger-Low', 'fear-Low', 'surprise-Low'): (('sadness-Low',),
  0.9261363636363636),
 ('anger-Low', 'fear-MidLow'): (('sadness-Low',), 0.927710843373494),
 ('anger-Low', 'joy-Mid'): (('sadness-Low',), 0.9166666666666666),
 ('anger-Low', 'joy-MidHigh'): (('sadness-Low',), 0.9627659574468085),
 ('anger-Low', 'joy-MidHigh', 'sadness-Low'): (('fear-Low',),
  0.7845303867403315),
 ('anger-Low', 'joy-MidHigh', 'sadness-Low', 'surprise-Low'): (('fear-Low',),
  0.75),
 ('anger-Low', 'joy-MidHigh', 'surprise-Low'): (('sadness-Low',),
  0.9645390070921985),
 ('anger-Low', 'surprise-Low'): (

In [ ]:
rules_df = pd.DataFrame.from_dict(rules, orient='index',columns=['Right','Confidence'])
rules_df.index = [', '.join(item) for item in rules_df.index]
rules_df['Right'] = [','.join(item) for item in rules_df['Right']]
rules_df['CountLeftSide'] = rules_df.index.str.count('-')
rules_df['CountRightSide'] = rules_df['Right'].str.count('-')
rules_df = rules_df[rules_df['CountLeftSide']<3]
rules_df = rules_df[rules_df['CountRightSide']==1]
rules_df

,Right,Confidence,CountLeftSide,CountRightSide
"anger-Low, joy-Mid",sadness-Low,0.916667,2,1
"joy-Mid, sadness-Low",anger-Low,1.000000,2,1
"anger-Low, fear-MidLow",sadness-Low,0.927711,2,1
"fear-MidLow, sadness-Low",anger-Low,0.993548,2,1
"joy-MidHigh, sadness-Low",anger-Low,1.000000,2,1
"anger-Low, joy-MidHigh",sadness-Low,0.962766,2,1
"anger-Low, surprise-MidLow",sadness-Low,0.914894,2,1
"sadness-Low, surprise-MidLow",anger-Low,0.994220,2,1
"anger-Low, fear-Low",sadness-Low,0.907336,2,1
"fear-Low, sadness-Low",anger-Low,0.991561,2,1


In [ ]:
#Lift Calulation

def support_count(rhs):
  count = 0
  #rhs = set(rhs)
  for j in data['items']:
    #j = set(j)
    if (rhs in j):
      count = count+1
  return count

rhs_support=[]
for i in rules_df['Right']:
  a=support_count(i)
  rhs_support.append(a/len(data))

rules_df['RHS_support'] = rhs_support

rules_df['lift']=rules_df['Confidence']/rules_df['RHS_support']
rules_df = rules_df.sort_values(by=['lift'],ascending=False)
rules_df[0:10]

,Right,Confidence,CountLeftSide,CountRightSide,RHS_support,lift
"anger-Low, joy-MidHigh",sadness-Low,0.962766,2,1,0.898239,1.071837
"anger-Low, fear-MidLow",sadness-Low,0.927711,2,1,0.898239,1.032811
"anger-Low, joy-Mid",sadness-Low,0.916667,2,1,0.898239,1.020516
"anger-Low, surprise-MidLow",sadness-Low,0.914894,2,1,0.898239,1.018542
"anger-Low, fear-Low",sadness-Low,0.907336,2,1,0.898239,1.010128
"joy-Mid, sadness-Low",anger-Low,1.000000,2,1,0.990215,1.009881
"joy-MidHigh, sadness-Low",anger-Low,1.000000,2,1,0.990215,1.009881
"sadness-Low, surprise-MidLow",anger-Low,0.994220,2,1,0.990215,1.004044
"fear-MidLow, sadness-Low",anger-Low,0.993548,2,1,0.990215,1.003366
"fear-Low, sadness-Low",anger-Low,0.991561,2,1,0.990215,1.001359
